In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import yaml

In [ ]:
from primaite.simulator.network.networks import arcd_uc2_network


In [ ]:
net = arcd_uc2_network()

In [ ]:
random_node = list(net.nodes.keys())[0]
f = net.nodes[random_node].file_system.create_file(file_name="testfile")

In [ ]:
f.describe_state()

In [ ]:
def test_file_observation():
    from primaite.simulator.sim_container import Simulation
    from primaite.simulator.network.hardware.nodes.computer import Computer
    from primaite.game.actor.observations import FileObservation

    sim = Simulation()
    pc = Computer(hostname="beep", ip_address="123.123.123.123", subnet_mask="255.255.255.0")
    sim.network.add_node(pc)
    f = pc.file_system.create_file(file_name="dog.png")

    dog_file_obs = FileObservation(where=['network','nodes',pc.uuid,'file_system'])
    print(sim.describe_state())
test_file_observation()

In [ ]:
from primaite.simulator.sim_container import Simulation
from primaite.simulator.network.hardware.nodes.computer import Computer
from primaite.game.actor.observations import FileObservation, FolderObservation

sim = Simulation()
pc = Computer(hostname="beep", ip_address="123.123.123.123", subnet_mask="255.255.255.0")
sim.network.add_node(pc)
f = pc.file_system.create_file(file_name="dog.png")

state = sim.describe_state()

dog_file_obs = FileObservation(where=['network','nodes',pc.uuid,'file_system', 'folders','root','files','dog.png'])
root_folder_obs = FolderObservation(where=['network','nodes',pc.uuid,'file_system', 'folders','root'],files=[dog_file_obs])
print(dog_file_obs(state))
print(root_folder_obs(state))

In [ ]:
dog_file_obs.space

In [ ]:
root_folder_obs.space

In [ ]:
state

In [ ]:
import yaml

In [ ]:
with open('example_config.yaml', 'r') as file:
    conf = yaml.safe_load(file)

In [ ]:
conf['simulation']

In [5]:
import yaml
from primaite.simulator.sim_container import Simulation
from primaite.simulator.network.hardware.nodes.computer import Computer
from primaite.simulator.network.hardware.nodes.server import Server
from primaite.simulator.network.hardware.nodes.switch import Switch
from primaite.simulator.network.hardware.nodes.router import Router



In [33]:
# import yaml

from primaite.simulator.network.hardware.nodes.router import ACLAction
from primaite.simulator.network.transmission.network_layer import IPProtocol
from primaite.simulator.network.transmission.transport_layer import Port


class PrimaiteSession:

    def __init__(self):
        self.simulation: Simulation
        self.agents = []

    @classmethod
    def from_config(cls, cfg_path):
        ref_to_uuid = {}

        game = cls()
        with open(cfg_path, 'r') as file:
            conf = yaml.safe_load(file)
        
        #1. create nodes 
        sim = Simulation()
        net = sim.network
        nodes_cfg = conf['simulation']['network']['nodes']
        links_cfg = conf['simulation']['network']['links']
        for node_cfg in nodes_cfg:
            ref = node_cfg['ref']
            n_type = node_cfg['type']
            if n_type == 'computer':
                new_node = Computer(hostname = node_cfg['hostname'], 
                                    ip_address = node_cfg['ip_address'], 
                                    subnet_mask = node_cfg['subnet_mask'], 
                                    default_gateway = node_cfg['default_gateway'],
                                    dns_server = node_cfg['dns_server'])
            elif n_type == 'server':
                new_node = Server(hostname = node_cfg['hostname'], 
                                    ip_address = node_cfg['ip_address'], 
                                    subnet_mask = node_cfg['subnet_mask'], 
                                    default_gateway = node_cfg['default_gateway'],
                                    dns_server = node_cfg.get('dns_server'))
            elif n_type == 'switch':
                new_node = Switch(hostname = node_cfg['hostname'],
                                  num_ports = node_cfg.get('num_ports'))
            elif n_type == 'router':
                new_node = Router(hostname=node_cfg['hostname'],
                                  num_ports = node_cfg.get('num_ports'))
                if 'ports' in node_cfg:
                    for port_num, port_cfg in node_cfg['ports'].items():
                        new_node.configure_port(port=port_num, 
                                                ip_address=port_cfg['ip_address'],
                                                subnet_mask=port_cfg['subnet_mask'])
                if 'acl' in node_cfg:
                    for r_num, r_cfg in node_cfg['acl'].items():
                        new_node.acl.add_rule(
                            action = ACLAction[r_cfg['action']],
                            src_port = Port[r_cfg.get('port')],
                            dst_port = Port[r_cfg.get('port')],
                            protocol = IPProtocol[r_cfg.get('protocol')],
                            src_ip = r_cfg.get('ip_address'),
                            dst_ip = r_cfg.get('ip_address'),
                            position = r_num
                        )


            try:
                net.add_node(new_node)
                ref_to_uuid[ref] = new_node.uuid
            except BaseException:
                print(3)


        #2. start/setup simulation objects
        #3. create agents
        #4. set up agents' actions and observation spaces.
        game.simulation = sim
        return game

s = PrimaiteSession.from_config('example_config.yaml')
# print(s.simulation.describe_state())

2023-09-25 17:38:39,385: Added node b5486651-1c6f-449a-8019-6a3641cfb998 to Network 7c6e4724-1653-4db7-9bd0-44e8e380f1a1
2023-09-25 17:38:39,391: Added node 1533c2f7-389e-4e03-95b3-9cf059086490 to Network 7c6e4724-1653-4db7-9bd0-44e8e380f1a1
2023-09-25 17:38:39,395: Added node 6b6c3b24-61d4-46ac-9364-11d726e50ccb to Network 7c6e4724-1653-4db7-9bd0-44e8e380f1a1
2023-09-25 17:38:39,398: Added node a0bee8d0-2ab8-4e29-9a2c-23c6757b240c to Network 7c6e4724-1653-4db7-9bd0-44e8e380f1a1
2023-09-25 17:38:39,401: Added node 7cb2c102-62ba-4859-94f2-5d724de38733 to Network 7c6e4724-1653-4db7-9bd0-44e8e380f1a1
2023-09-25 17:38:39,403: Added node bec38db7-520e-4044-93db-08308278d66f to Network 7c6e4724-1653-4db7-9bd0-44e8e380f1a1
2023-09-25 17:38:39,407: Added node ae0c2253-3ec8-48c3-b5d2-0b37c19c885d to Network 7c6e4724-1653-4db7-9bd0-44e8e380f1a1


3
3
3


In [31]:
print(s.simulation.describe_state())

{'uuid': 'ceeb8791-b140-43d0-b59e-c3c3f533309b', 'network': {'uuid': 'ff176601-4e1d-4f89-8db4-33c0598ee105', 'nodes': {'6b9afe70-913b-40ce-9cee-1ee3648e43ce': {'uuid': '6b9afe70-913b-40ce-9cee-1ee3648e43ce', 'hostname': 'client_1', 'operating_state': 2, 'NICs': {'108c797d-32ca-4e93-8476-6b13cda6cf37': {'uuid': '108c797d-32ca-4e93-8476-6b13cda6cf37', 'ip_adress': '192.168.10.21', 'subnet_mask': '255.255.255.0', 'mac_address': 'af:5f:0c:00:d3:63', 'speed': 100, 'mtu': 1500, 'wake_on_lan': False, 'enabled': False}}, 'file_system': {'uuid': '32062959-b2ed-4d24-b5a9-7e99b7ebfcfe', 'folders': {'root': {'uuid': '8876d59b-d46d-414d-9ae2-5e948f65b175', 'name': 'root', 'health_status': 1, 'files': {}, 'is_quarantined': False}}}, 'applications': {}, 'services': {}, 'process': {}}, '227d1fb7-fc64-4273-9817-0f32280a0859': {'uuid': '227d1fb7-fc64-4273-9817-0f32280a0859', 'hostname': 'client_2', 'operating_state': 2, 'NICs': {'22119571-b47d-4ffb-998c-62173c670f78': {'uuid': '22119571-b47d-4ffb-998c-6